In [10]:
from predict import PredictWNet
from config import Config
import tifffile as tiff
import numpy as np
import sys
import napari

sys.path.append("../..")
from utils import create_dataset_dict_no_labs

In [11]:
config = Config()
predictor = PredictWNet(
    trained_model_path=r"./chkpt_res/test_wnet_checkpoint_4500e.pth",
    config=config,
    crf=True,
)

train_files = create_dataset_dict_no_labs(
    volume_directory=r"../../dataset/cropped_visual/val/volumes"
)
train_files = [d.get("image") for d in train_files]
volumes = tiff.imread(train_files).astype(np.float32)

2023-04-11 14:14:19,830 - Images :
2023-04-11 14:14:19,836 - crop_vol_val
2023-04-11 14:14:19,837 - crop_vol_val2
2023-04-11 14:14:19,837 - **********


In [12]:
segmentations = predictor.predict_batch(volumes)
#segmentations = np.where(segmentations > 0.5, 0, 1)
tiff.imwrite(r"../../results/segmentations.tif", segmentations)

In [4]:
segmentations_max = np.argmax(segmentations, axis=1)
segmentation_0 = np.where(segmentations_max == 0, segmentations_max + 1, 0)
segmentation_1 = np.where(segmentations_max == 1, segmentations_max, 0)
segmentation_2 = np.where(segmentations_max == 2, segmentations_max - 1, 0)
print(segmentation_0.shape)

(2, 64, 64, 64)


In [8]:
viewer = napari.viewer.Viewer()
for i in range(segmentation_0.shape[0]):
    viewer.add_image(volumes[i], name=f"Original image {i}", colormap="inferno")
    # viewer.add_labels(segmentation_0[i], name=f"Segmentation 0 image {i}")
    # viewer.add_image(segmentation_0[i], name=f"Segmentation 1 image {i}")
    viewer.add_image(segmentation_1[i], name=f"Segmentation 1 image {i}")
    # viewer.add_image(segmentation_2[i], name=f"Segmentation 2 image {i}")

In [9]:
from post_processing import binary_watershed

viewer.add_labels(
    binary_watershed(viewer.layers.selection.active.data), name="watershed"
)

<Labels layer 'watershed' at 0x2142fb3c670>